# Segment and Cluster Toronto Neighborhoods

In [40]:
!conda install -c anaconda xlrd --yes
!conda install -c anaconda beautifulsoup4

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 anaconda

beautifulsoup4 100% |################################| Time: 0:00:00  40.64 MB/s


In [41]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Fetch the page content from Wikipedia

In [79]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = page.content

NOTE:

In the code below I make the following assumptions:

1. The table I'm looking has a specific class 'wikitable' and that there is only 1 table with that class. 
2. The data in the table is displayed in the order Postalcode, Borough, Neighborhood.
3. All tables have a value (including "Not assigned")

Having inspected the HTML of the page I clarified that the above assumptions were true at the time of creation (March 16th, 2019)

----

Use BeautifulSoup to help scrape data from the returned Wikipedia page content

Docs: https://www.crummy.com/software/BeautifulSoup/

In [85]:
soup = BeautifulSoup(contents, 'html.parser')

headers = ['Postcode', 'Borough', 'Neighborhood']

table = soup.find('table',{'class':'wikitable'})
table_rows = table.find_all('tr')
table_rows = table_rows[1:]

df_rows = []

for row in table_rows:
    items = row.find_all('td')
    if items[1].text.strip() != 'Not assigned':
        df_row =[]
        df_row.append(items[0].text.strip())
        df_row.append(items[1].text.strip())
        df_row.append(items[2].text.strip())
        df_rows.append(df_row)


In [90]:
init_df = pd.DataFrame(data=df_rows, columns=headers)
init_df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Now that we have the initial dataframe we need to clean up the data by doing the following:

1. Combine Neighborhoods with the same Postcode
2. Set any Neighborhood with the value of "Not assigned" to be the same as the Borough

In the code below I loop over the rows of the dataframe and create a unique mapping of each postal code. During this process I concatenate the Neighborhoods so that each unique postal code row has a string containing all the Neighborhoods associated with it.

In [148]:
init_df.groupby(['Postcode']).head()

c_data = {} # cleaned data mapping

for index, row in init_df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    if not row['Postcode'] in c_data:
        c_data[row['Postcode']] = [row['Postcode'], row['Borough'], row['Neighborhood']]
    elif not row['Neighborhood'] in c_data[row['Postcode']][2] :
        c_data[row['Postcode']][2] += ", " + row['Neighborhood']

In [169]:
clean_df = pd.DataFrame(list(c_data.values()), columns=headers)

In [170]:
clean_df.head(11)

,Postcode,Borough,Neighborhood
0,M1E,Scarborough,"Guildwood, Morningside, West Hill"
1,M2N,North York,Willowdale South
2,M7R,Mississauga,Canada Post Gateway Processing Centre
3,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
4,M1G,Scarborough,Woburn
5,M4B,East York,"Woodbine Gardens, Parkview Hill"
6,M4K,East Toronto,"The Danforth West, Riverdale"
7,M1X,Scarborough,Upper Rouge
8,M5S,Downtown Toronto,"Harbord, University of Toronto"
9,M5E,Downtown Toronto,Berczy Park


In [172]:
clean_df.shape

(103, 3)